# Data Pipeline Experimentation

In [8]:
import os
import sys

parent_dir = os.path.abspath("/home/leandre/Projects/BioMoQA_Playground")

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from src.data_pipeline.ipbes.create_ipbes_raw import loading_pipeline_from_raw

In [ ]:
datasets=loading_pipeline_from_raw(multi_label=True)